In [1]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('추자도.csv', encoding='cp949')

df = df.rename(columns={'일시': 'date', '평균기압(hPa)': 'Apress(hPa)', '평균 기온(°C)': 'Atemp(°C)', '평균 수온(°C)': 'Stemp(°C)', '평균 상대습도(%)': 'Moisture(%)'})
df=df.set_index(pd.to_datetime(df.date)) # Date를 인덱스로
df=df[['Stemp(°C)']] # Column 재정리
df.dropna(inplace=True)
df

,Stemp(°C)
date,
2014-01-16,14.6
2014-01-17,14.8
2014-01-18,14.9
2014-01-19,14.8
2014-01-20,14.6
...,...
2022-12-27,16.1
2022-12-28,16.0
2022-12-29,16.1


In [34]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

# Step 1: Explore and preprocess the data
# Assuming the given data frame is stored in a variable called 'df'

# Normalize the temperature values
min_temp = df['Stemp(°C)'].min()
max_temp = df['Stemp(°C)'].max()
df['normalized_temp'] = (df['Stemp(°C)'] - min_temp) / (max_temp - min_temp)

# Step 2: Split the data
# Assuming you want to use the last 30% of the data as the testing set
split_index = int(len(df) * 0.7)
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

# Step 3: Create the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])
        return output

# Step 4: Train the LSTM model
input_size = 1  # Number of features (in this case, we only have temperature as the input)
hidden_size = 32  # Number of hidden units in the LSTM layer
output_size = 1  # Number of output units (predicting temperature)
num_epochs = 100
learning_rate = 0.001

# Convert the training data to PyTorch tensors
train_inputs = torch.Tensor(train_data['normalized_temp'].values).view(-1, 1, 1)
train_targets = torch.Tensor(train_data['normalized_temp'].values).view(-1, 1, 1)

# Initialize the LSTM model
model = LSTMModel(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(train_inputs)
    loss = criterion(outputs, train_targets)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# End of step 4



c:\Users\admin\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2032, 1, 1])) that is different to the input size (torch.Size([2032, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/100], Loss: 0.1233
Epoch [20/100], Loss: 0.1034
Epoch [30/100], Loss: 0.0857
Epoch [40/100], Loss: 0.0708
Epoch [50/100], Loss: 0.0592
Epoch [60/100], Loss: 0.0517
Epoch [70/100], Loss: 0.0478
Epoch [80/100], Loss: 0.0467
Epoch [90/100], Loss: 0.0466
Epoch [100/100], Loss: 0.0466


In [35]:
# Step 5: Model Validation and Future Temperature Prediction
model.eval()

# Convert the testing data to PyTorch tensors
test_inputs = torch.Tensor(test_data['normalized_temp'].values).view(-1, 1, 1)

# Perform predictions on the testing set
with torch.no_grad():
    test_predictions = model(test_inputs)

# Denormalize the predicted values
test_predictions = (test_predictions * (max_temp - min_temp)) + min_temp

# Calculate evaluation metrics
mse = criterion(test_predictions, torch.Tensor(test_data['Stemp(°C)'].values).view(-1, 1))
rmse = torch.sqrt(mse)
mae = torch.mean(torch.abs(test_predictions - torch.Tensor(test_data['Stemp(°C)'].values).view(-1, 1)))

print(f"Mean Squared Error (MSE): {mse.item():.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse.item():.4f}")
print(f"Mean Absolute Error (MAE): {mae.item():.4f}")

Mean Squared Error (MSE): 23.7826
Root Mean Squared Error (RMSE): 4.8767
Mean Absolute Error (MAE): 4.1152


In [37]:
# Assuming you have the trained LSTM model, 'model', and the necessary data

# Get the temperature value for the last day of the available data
previous_temp = torch.Tensor([df.iloc[-1]['normalized_temp']]).view(1, 1, 1)

# Set the model to evaluation mode
model.eval()

# Define the number of days to predict (e.g., 30 days for one month)
num_days = 30

# Predict the temperature for each subsequent day
for _ in range(num_days):
    with torch.no_grad():
        future_prediction = model(previous_temp)
    
    # Denormalize the predicted value
    denormalized_prediction = (future_prediction * (max_temp - min_temp)) + min_temp
    
    # Print or store the predicted temperature for the current day
    print(f"Predicted temperature: {denormalized_prediction.item():.2f} °C")
    
    # Update the 'previous_temp' for the next iteration
    previous_temp = future_prediction.view(1, 1, 1)

Predicted temperature: 18.20 °C
Predicted temperature: 18.36 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
Predicted temperature: 18.37 °C
